# Inversão do Te  do Havaí

In [ ]:
from __future__ import division, print_function
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
x, topo = np.loadtxt('../DiaDois/Flexura/Oahu_profile.txt', unpack=True)

In [ ]:
plt.plot(x, topo, '-k')

## Desafio: Estimar o regional com uma parábola

In [ ]:
from fatiando.inversion import Misfit

In [ ]:
class Parabola(Misfit):
    
    def __init__(self, x, y):
        super(Parabola, self).__init__(data=y, nparams=3, islinear=True)
        self.x = x
        
    def predicted(self, p):
        a, b, c = p
        return a*self.x**2 + b*self.x + c
    
    def jacobian(self, p):
        A = np.empty((self.ndata, self.nparams))
        A[:, 0] = self.x**2
        A[:, 1] = self.x
        A[:, 2] = 1
        return A

In [ ]:
par = Parabola(x, topo).fit()

In [ ]:
par.estimate_

In [ ]:
plt.plot(x, topo, '-k')
plt.plot(x, par.predicted(), '-r')

## Ajuste robusto

In [ ]:
par_robusto = Parabola(x, topo).fit()

In [ ]:
r = par_robusto.residuals()

In [ ]:
plt.plot(x, topo, '-k')
plt.plot(x, r, '-r')

In [ ]:
pesos = 1/np.abs(r)
pesos[r < 1e-5] = 1e5

In [ ]:
plt.plot(x, topo, '-k')
plt.plot(x, r, '-r')

In [ ]:
plt.plot(x, pesos, '-b')

In [ ]:
for i in xrange(20):
    r = par_robusto.residuals()
    pesos = 1/np.abs(r)
    pesos[np.abs(r) < 1e-5] = 1e5
    par_robusto.set_weights(pesos).fit()

In [ ]:
plt.plot(x, topo, '-k')
plt.plot(x, par.predicted(), '-r')
plt.plot(x, par_robusto.predicted(), '-b')

In [ ]:
topo_res = topo - par_robusto.predicted()

In [ ]:
plt.plot(x, topo_res, '-k')

## Desafio: Inversão para o Te usando flexura

In [ ]:
from scipy import sparse
from scipy.sparse.linalg import dsolve

In [ ]:
class FlexNum3(object):    
    drho = 2300.0
    g = 10.0
    E = 1.0E11
    nu = 0.25
    V0 = -1.0E13    
    
    def __init__(self, x, p):
        self.x = x
        self.p = p
        self.dx = x[1]-x[0]
        self.N = p.size
        
    def monta_roda(self, Te):
        D = self.E*Te**3/(12*(1-self.nu**2))
        N,dx,drho,g = self.N,self.dx,self.drho,self.g
        
        Alin = np.arange(N)
        Alin = np.append(Alin,np.arange(0,N-1))
        Alin = np.append(Alin,np.arange(1,N))
        Alin = np.append(Alin,np.arange(0,N-2))
        Alin = np.append(Alin,np.arange(2,N))
        
        Acol = np.arange(N)
        Acol = np.append(Acol,np.arange(1,N))
        Acol = np.append(Acol,np.arange(0,N-1))
        Acol = np.append(Acol,np.arange(2,N))
        Acol = np.append(Acol,np.arange(0,N-2))
        
        Aval = np.zeros(N) + 6*D + dx**4*drho*g
        Aval = np.append(Aval,np.zeros(N-1)-4*D)
        Aval = np.append(Aval,np.zeros(N-1)-4*D)
        Aval = np.append(Aval,np.zeros(N-2)+D)
        Aval = np.append(Aval,np.zeros(N-2)+D)
        
        self.A = sparse.csr_matrix((Aval,(Alin,Acol)), shape=(N,N))
        
        return dsolve.spsolve(self.A,self.p*dx**4)

In [ ]:
h_load = np.copy(topo_res)
ombreira = (x < 510) | (x > 649)
h_load[ombreira] = 0

plt.plot(x, h_load, '-k')

In [ ]:
p = -h_load*(2700-1000)*10
flex = FlexNum3(x*1000, p)

In [ ]:
w = flex.monta_roda(70e3)

In [ ]:
plt.plot(x, topo_res, '-k')
plt.plot(x, w, '-r')

In [ ]:
class Dummy(object):
    def __init__(self, *args):
        pass
    def hard_reset(self):
        pass

In [ ]:
class FlexTe(Misfit):
    
    def __init__(self, x, topo, flex):
        self.hessian = Dummy()
        super(FlexTe, self).__init__(data=topo, nparams=1, islinear=False)
        self.x = x
        self.flex = flex
        
    def predicted(self, p):
        w = self.flex.monta_roda(p[0])
        return w
    
    def jacobian(self, p):
        A = np.empty((self.ndata, 1))
        dt = 1
        A[:, 0] = (self.flex.monta_roda(p[0] + dt) - self.flex.monta_roda(p[0] - dt))/dt*2
        return A
        

In [ ]:
solver = FlexTe(x, topo_res, flex)

In [ ]:
solver.config('acor', bounds=[30e3, 150e3]).fit()

In [ ]:
solver.estimate_

In [ ]:
plt.plot(x, topo_res, '-k')
plt.plot(x, solver.predicted(), '-r')

In [ ]:
peso = np.ones(solver.ndata)
peso[~ombreira] = 0
solver.set_weights(peso)

In [ ]:
solver.fit()

In [ ]:
solver.estimate_

In [ ]:
plt.plot(x, topo_res, '-k')
plt.plot(x, solver.predicted(), '-r')